In [3]:
from __future__ import absolute_import, division, print_function
from builtins import (
    ascii, bytes, chr, dict, filter, hex, input, int, map,
    next, oct, open, pow, range, round, str, super, zip)

import os
import json

import psycopg2
import gdal
import osgeo
import geopandas as gp
import numpy as np
import pandas as pd

In [94]:
cong_df = pd.read_excel(
    'data/raw/2016_all/wi_2016_congress.xlsx', sheetname=list(range(1, 9)))

for key in cong_df:
    cong_df[key]['WEC Canvass Reporting System'].fillna(method='pad', inplace=True)
    cong_df[key]['Unnamed: 3'].fillna(method='bfill', inplace=True)
    cong_df[key]['Unnamed: 4'].fillna(method='bfill', inplace=True)
    cong_df[key]['Unnamed: 5'].fillna(method='bfill', inplace=True)
    cong_df[key] = cong_df[key][[0, 1, 3, 4]]
    cols = list(cong_df[key].iloc[0])
    cols[0] = 'COUNTY'
    cols[1] = 'WARDS'
    rename_dict = {
        'WEC Canvass Reporting System': cols[0],
        'Unnamed: 1': cols[1],
        'Unnamed: 3': cols[2],
        'Unnamed: 4': cols[3]
    }
    cong_df[key].rename(columns=rename_dict, inplace=True)
    for party in {'REP', 'DEM'}:
        if not party in cong_df[key].columns:
            cong_df[key][party] = 0
    for col_name in cong_df[key].columns:
        if not col_name in {'REP', 'DEM', 'COUNTY', 'WARDS'}:
            cong_df[key].drop(col_name, axis=1, inplace=True)
    cong_df[key].dropna(inplace=True)
    cong_df[key]

In [95]:
cong_df[4]

,COUNTY,WARDS,DEM,REP
9,MILWAUKEE,"VILLAGE OF BAYSIDE Wards 1,3",557,0
10,MILWAUKEE,"VILLAGE OF BAYSIDE Wards 2,4",705,0
11,MILWAUKEE,VILLAGE OF BAYSIDE Ward 5,355,0
12,MILWAUKEE,VILLAGE OF BROWN DEER Wards 1-2,1565,0
13,MILWAUKEE,VILLAGE OF BROWN DEER Wards 3-4,1334,0
14,MILWAUKEE,VILLAGE OF BROWN DEER Wards 5-6,1443,0
15,MILWAUKEE,VILLAGE OF FOX POINT Wards 1-4,1036,0
16,MILWAUKEE,VILLAGE OF FOX POINT Wards 5-9,1455,0
17,MILWAUKEE,VILLAGE OF SHOREWOOD Ward 1-4,2238,0
18,MILWAUKEE,VILLAGE OF SHOREWOOD Ward 5-8,1883,0


In [ ]:
cong_d